In [3]:
import os

In [100]:
!pip install pandas

In [19]:
import pandas as pd

In [4]:
# !pip install psycopg2
# import psycopg2

In [5]:
# %load_ext sql

In [6]:
HOST = "localhost"
DB = "tweetsDB"
USER = "postgres"
PASSWD = "passwd"

In [7]:
# conn = psycopg2.connect(dbname=DB, user=USER, password=PASSWD, host=HOST)

In [8]:
# cur = conn.cursor()

In [9]:
query = """
    SELECT value 
    FROM hashtags
    LIMIT 100
"""

In [10]:
# cur.execute(query)

In [11]:
# cur.fetchall()

In [12]:
# res = cur.execute("SELECT * FROM hashtags LIMIT 100")

In [13]:
# print(res)

In [14]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kdeni\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
sid = SentimentIntensityAnalyzer()

In [16]:
from sqlalchemy import create_engine

In [17]:
posgres_conn_str = f"postgresql://{USER}:{PASSWD}@{HOST}/{DB}"

In [18]:
engine = create_engine(posgres_conn_str)

TODO:

    1. vytvorit si novy stlpec pre sentiment(neg, neu, pos, compound)
    2. joinut tweety s tweethashtagmi s hashtagmi 
    3. vyfiltrovat podla danych hashtagov, ktore chcem 
    4. vypocitat sentiment a zapisat do DB

In [20]:
df = pd.read_sql(query, engine)

In [72]:
string = "Hovmp"

In [73]:
string.lower()

'hovmp'

In [51]:
HASHTAG_FILTER = """'DeepstateVirus', 'DeepStateVaccine',
'DeepStateFauci', 'QAnon', 'Agenda21', 'CCPVirus', 'ClimateChangeHoax',
'GlobalWarmingHoax', 'ChinaLiedPeopleDied', 'SorosVirus', '5GCoronavirus', 'MAGA',
'WWG1WGA', 'Chemtrails', 'flatEarth', 'MoonLandingHoax', 'moonhoax', 'illuminati',
'pizzaGateIsReal', 'PedoGateIsReal', '911truth', '911insidejob', 'reptilians'"""

In [101]:
CONSPIRACY_THEORIES = {
        "Deep State"                            : "DeepstateVirus, DeepStateVaccine, DeepStateFauci",
        "Qanon"                                 : "QAnon, MAGA, WWG1WGA",
        "New world order"                       : "Agenda21",
        "The virus escaped from a Chinese lab"  : "CCPVirus, ChinaLiedPeopleDied",
        "GLobal Warming is HOAX"                : "ClimateChangeHoax, GlobalWarmingHoax",
        "COVID19 and microchipping"             : "SorosVirus, BillGAtes",
        "COVID19 is preaded by 5G"              : "5GCoronavirus",
        "Moon landing is fake"                  : "MoonLandingHoax, moonhoax",
        "9/11 was inside job"                   : "911truth, 911insidejob",
        "Pizzagate conspiracy theory"           : "pizzaGateIsReal, PedoGateIsReal",
        "Chemtrails"                            : "Chemtrails",
        "FlatEarth"                             : "flatEarth",
        "Illuminati"                            : "illuminati",
        "Reptilian conspiracy theory"           : "reptilians"
}

We need to match all the mutations of the hashtags:

In [74]:
filter_query = f"""
    SELECT * 
    FROM hashtags
    WHERE lower(value) LIKE ANY (array[{HASHTAG_FILTER.lower()}])
"""

In [75]:
filter_query

"\n    SELECT * \n    FROM hashtags\n    WHERE lower(value) LIKE ANY (array['deepstatevirus', 'deepstatevaccine',\n'deepstatefauci', 'qanon', 'agenda21', 'ccpvirus', 'climatechangehoax',\n'globalwarminghoax', 'chinaliedpeopledied', 'sorosvirus', '5gcoronavirus', 'maga',\n'wwg1wga', 'chemtrails', 'flatearth', 'moonlandinghoax', 'moonhoax', 'illuminati',\n'pizzagateisreal', 'pedogateisreal', '911truth', '911insidejob', 'reptilians'])\n"

In [97]:
qanon_query = f"""
    SELECT * 
    FROM hashtags
    WHERE lower(value) LIKE ANY (array['qanon'])
"""

In [98]:
df_qanon = pd.read_sql(qanon_query, engine)

In [76]:
df_filtered = pd.read_sql(filter_query, engine)

Count of every mutation of the "conspiracy" hashtags

In [77]:
df_filtered

,id,value
0,111971,agenda21
1,363281,AGENDA21
2,905802,ChemTrails
3,3907585,SorosVirus
4,3486140,GlobalWarmingHoax
...,...,...
125,22849188,QanOn
126,20506,QAnon
127,77112,Qanon
128,25089,WWG1WGA


In [99]:
df_qanon

,id,value
0,15168832,QANOn
1,15170088,qaNon
2,15189267,QAnoN
3,15274024,QaNON
4,15752301,QANon
5,15856421,qAnoN
6,18671401,qAnON
7,16204212,QaNon
8,18149870,qanoN
9,16070849,qaNON


We can see that hoaxer dont know grammar

In [68]:
splited = hastag_filter.split(sep=",")

In [69]:
splited

["'DeepstateVirus'",
 " 'DeepStateVaccine'",
 "\n'DeepStateFauci'",
 " 'QAnon'",
 " 'Agenda21'",
 " 'CCPVirus'",
 " 'ClimateChangeHoax'",
 "\n'GlobalWarmingHoax'",
 " 'ChinaLiedPeopleDied'",
 " 'SorosVirus'",
 " '5GCoronavirus'",
 " 'MAGA'",
 "\n'WWG1WGA'",
 " 'Chemtrails'",
 " 'flatEarth'",
 " 'MoonLandingHoax'",
 " 'moonhoax'",
 " 'illuminati'",
 "\n'pizzaGateIsReal'",
 " 'PedoGateIsReal'",
 " '911truth'",
 " '911insidejob'",
 " 'reptilians'"]

Next we will perform the same query but also with joins

After that we will need to parse all the tweet contents and compute the sentiment